# Setup

In [2]:
!git clone https://github.com/autonlab/AutoFeat
!python -m pip install -r ./AutoFeat/requirements.txt

Cloning into 'AutoFeat'...
remote: Enumerating objects: 1082, done.
remote: Counting objects: 100% (1082/1082), done.
remote: Compressing objects: 100% (375/375), done.
remote: Total 1082 (delta 738), reused 1032 (delta 688), pack-reused 0
Receiving objects: 100% (1082/1082), 132.58 KiB | 5.10 MiB/s, done.
Resolving deltas: 100% (738/738), done.
You should consider upgrading via the '/Users/dhruvsrikanth/Work/CMU/AutoFeat/.venv_test/bin/python -m pip install --upgrade pip' command.


In [3]:
import sys
sys.path.append('AutoFeat')

# Import Libraries

In [11]:
import pandas as pd
import numpy as np
import sklearn
import autofeat as aft

# Prepare Dataset

In [43]:
df = pd.read_csv('datasets/shampoo_sales.csv')
df.head()

,Month,Sales
0,1-01,266.0
1,1-02,145.9
2,1-03,183.1
3,1-04,119.3
4,1-05,180.3


In [ ]:
# Define the target variable to be sales for the next month (t+1)
outcome = df['Sales'].shift(-1)
outcome.head()

In [44]:
# Create X and y (Use sales from the previous month to predict sales for the current month)
X = df.drop(['Sales'], axis=1)
y = outcome

0    145.9
1    183.1
2    119.3
3    180.3
4    168.5
Name: Sales, dtype: float64

# Setup Multi-Feature Extractor Pipeline

## Sliding Window

In [36]:
# Sliding window parameters
window_size = 10
step_size = 1

# Create sliding window
window = aft.SlidingWindow(window_size=window_size, step_size=step_size)

## Multi-Feature Extractor

In [37]:
# Define the transforms
transforms = [
    aft.MeanTransform(),
    aft.VarTransform(),
    aft.MinTransform(),
    aft.MaxTransform(),
    aft.MedianTransform(),
]

## Pipeline

In [38]:
def get_features(data, transforms, window):
    """
    Extract features from data using the given transforms.

    Parameters
    ----------
    data : pandas.DataFrame
        Data to extract features from.
    
    transforms : List(aft.Transform)
        List of transforms to apply to data.
    
    window : aft.SlidingWindow
        Sliding window to apply to data.
    
    Returns
    -------
    features : pandas.DataFrame
        Dataframe containing extracted features.
    """
    features = pd.DataFrame()

    # Iterate over each transform
    for transform in transforms:
        # Get the featurizer
        featurizer = window.use(transform)

        # Iterate over each column
        for col in data.columns:
            # Extract feature
            feature = featurizer(data[col].to_numpy())

            # Add feature to dataframe
            features[col + "_" + transform.get_name()] = feature

    return features

# Extract Features

In [39]:
extracted_features = get_features(data=df.iloc[:, 1:], transforms=transforms, window=window)
extracted_features.head()

,Sales_Mean,Sales_Variance,Sales_Min,Sales_Max,Sales_Median
0,183.51,2034.9789,119.3,266.0,181.70
1,190.56,3645.4104,119.3,336.5,181.70
2,194.56,3432.1304,119.3,336.5,184.50
3,195.68,3417.7496,119.3,336.5,189.35
4,198.70,3038.4980,122.9,336.5,189.35


Concatenate the extracted features with the original data.

In [41]:
df_features = pd.concat([df, extracted_features], axis=1)
df_features.head()

,Month,Sales,Sales_Mean,Sales_Variance,Sales_Min,Sales_Max,Sales_Median
0,1-01,266.0,183.51,2034.9789,119.3,266.0,181.70
1,1-02,145.9,190.56,3645.4104,119.3,336.5,181.70
2,1-03,183.1,194.56,3432.1304,119.3,336.5,184.50
3,1-04,119.3,195.68,3417.7496,119.3,336.5,189.35
4,1-05,180.3,198.70,3038.4980,122.9,336.5,189.35


# Fit Data

## Split Data

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

## Fit Model

In [45]:
model = sklearn.linear_model.LinearRegression()

AttributeError: module 'sklearn' has no attribute 'linear_model'

In [46]:
model.fit(X_train, y_train)

NameError: name 'model' is not defined